# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto


## 1. Download data from the Wikipedia page
We'll use the beautifulsoup package to download the table

In [21]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [22]:
wiki_page_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

wiki_page_text = requests.get(wiki_page_url).text

In [27]:
soup = BeautifulSoup(wiki_page_text,'lxml')

table = soup.find('table',{'class':'wikitable sortable'})

each_row = table.text.split('\n')

In [54]:
index = 0
output = [[]]
for row in each_row:
    if row != "":
        output[index].append(row)
    if len(output[index]) == 3:
        index += 1
        output.append([])

In [60]:
df = pd.DataFrame(output)

header = df.iloc[0]
df = df[1:]
df.columns = header